In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors

# Load data

In [3]:
articles = pd.read_csv("../data/texty_news.csv", sep='\t')
articles.head()

,title,article
0,60 хвилин на реакцію. Як ефективно протидіяти ...,Одрі Танг любить висловлюватися точно. Під час...
1,Фільм про нас. Cеріал «Чорнобиль» показує вади...,"«Влада погано комунікує з суспільством», – чує..."
2,Київські візерунки. Зіграйте у гру й перевірте...,"Що бачить художник, коли дивиться на карту Киє..."
3,Протистояння медіа та соцмереж під час виборів...,Модераторка дискусії Діана Дуцик з Могилянсько...
4,Філарет проти канонічності. Як українські прав...,"Попри це, патріархові Філарету потрібно віддат..."


In [4]:
articles.isnull().sum()

title        0
article    219
dtype: int64

In [0]:
articles = articles.dropna()
articles = articles.reset_index(drop=True)

In [6]:
articles.head()

,title,article
0,60 хвилин на реакцію. Як ефективно протидіяти ...,Одрі Танг любить висловлюватися точно. Під час...
1,Фільм про нас. Cеріал «Чорнобиль» показує вади...,"«Влада погано комунікує з суспільством», – чує..."
2,Київські візерунки. Зіграйте у гру й перевірте...,"Що бачить художник, коли дивиться на карту Киє..."
3,Протистояння медіа та соцмереж під час виборів...,Модераторка дискусії Діана Дуцик з Могилянсько...
4,Філарет проти канонічності. Як українські прав...,"Попри це, патріархові Філарету потрібно віддат..."


In [7]:
example_article = articles.article[1]
example_title = articles.title[1]

print("Title: " + example_title)
print("Article: " + example_article)

Title: Фільм про нас. Cеріал «Чорнобиль» показує вади радянської системи, дотепер вкорінені в Україні
Article: «Влада погано комунікує з суспільством», – чуємо ми сьогодні. У цього явища багато причин, і одна з них криється у Чорнобильській трагедії. Аварію, причини й наслідки якої попервах погано розуміли навіть у Кремлі, усіляко приховували, применшували. Власне, для СРСР це було «доброю традицією». ЧАЕС не була першою техногенною катастрофою (варто задати хоча б Киштимську аварію на Уралі 1957-го).  Однак масштаб та інформаційний ефект Чорнобиля – нечуваний. Відтоді серед українців залишилася помітна травма: коли трапляється щось надзвичайне, влада вочевидь брехатиме і викривлятиме інформацію. Тому в нашому соціумі є дуже велике поле для чуток і маніпуляцій.  Вже в добу соцмереж нерідко з’являються фейкові повідомлення в стилі «чоловік моєї подруги працює пожежником у Чорнобильські зоні, і він сказав»… – а далі йдуть різні варіації жахалок. Це «заходить» і через 30 років після вибух

# Clean data

In [0]:
from spacy.lang.uk.stop_words import STOP_WORDS

In [0]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    pattern = re.compile('[\W_]+')
    text = pattern.sub(' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = STOP_WORDS
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [10]:
clean_text(example_article)

'влада погано комунікує суспільством чуємо явища причин одна криється чорнобильській трагедії аварію причини наслідки попервах погано розуміли кремлі усіляко приховували применшували власне срср доброю традицією чаес першою техногенною катастрофою варто задати киштимську аварію уралі 1957 го масштаб інформаційний ефект чорнобиля нечуваний відтоді серед українців залишилася помітна травма трапляється надзвичайне влада вочевидь брехатиме викривлятиме інформацію соціумі велике поле чуток маніпуляцій добу соцмереж являються фейкові повідомлення стилі чоловік подруги працює пожежником чорнобильські зоні йдуть різні варіації жахалок заходить 30 вибуху аес надто цинічною брехня брехатимуть впевнені мільйони громадян незалежній україні тотальної цензури всемогутнього кгб імперських амбіцій чиновникам надзвичайних ситуацій доводиться докладати чимало зусиль викликати бодай якусь довіру порівняно недавня історія 2007 го села ожидів львівщині перекинулися цистерни фосфором генерал олександр кузьм

In [11]:
# Clean the summaries and texts
clean_summaries = []
for summary in articles.title:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in articles.article:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [12]:
# Example of cleaned article
print(clean_summaries[0])
print()
print(clean_texts[0])

60 хвилин на реакцію як ефективно протидіяти дезінформації не запроваджуючи цензуру досвід тайваню

одрі танг любить висловлюватися точно інтерв ю тайванський міністр портфеля табличці іменем танга написано цифровий міністр негайно виправляє згадуємо термін фейкові новини прийнятний термін дезінформація танг юридичне поняття тайвані тобто навмисна спрямована шкоду неправда найважливіше шкодить громадськості демократичній системі образу міністра сміхом шкодити міністру звичайна якісна журналістика відміну владних режимів азії таких сингапур тайвань вирішив боротися пошестю дезінформації вдаючись цензури арештів тайвань вийшов воєнного стану 1987 провів перші президентські вибори 1996 му попри економіка тайваню суттєво залежить торгівлі інвестицій китаю вважає тайвань своєю територією демократія продовжувала відокремлювати політичної системи континенті виборці тайваню обирають партією гоміндан схильна тісніших стосунків китаєм демократичною прогресивною партією дпп наголошує автономії пр

## Count the number of occurrences of each word in a set of text

In [0]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [14]:
mydict = {}
count_words(mydict, ["ми будемо їсти піццу сьогодні ввечері", "їсти овочі корисно"])
mydict

{'будемо': 1,
 'ввечері': 1,
 'корисно': 1,
 'ми': 1,
 'овочі': 1,
 'піццу': 1,
 'сьогодні': 1,
 'їсти': 2}

In [15]:
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 158085


# Load embeddings for Ukrainian language

To run this, you need to download the embedding into the folder 'deep_learning'

In [16]:
embeddings_index = {}
with open('news.lowercased.tokenized.word2vec.300d') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 328959


In [17]:
# explore e,bedding dimention
embeddings_index["мама"].shape

(300,)

In [18]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from Word2Vec:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from Word2Vec: 61
Percent of words that are missing from vocabulary: 0.04%


In [19]:
missing_words = []
for word, count in word_counts.items():
    if count > threshold and word not in embeddings_index:
        missing_words.append((word,count))
missing_words

[('гриняка', 22),
 ('австро', 33),
 ('none', 161),
 ('width', 232),
 ('parent', 53),
 ('container', 159),
 ('height', 67),
 ('5em', 53),
 ('bottom', 106),
 ('7em', 53),
 ('font', 318),
 ('weight', 106),
 ('align', 106),
 ('18px', 106),
 ('1em', 53),
 ('verdana', 53),
 ('arial', 54),
 ('serif', 53),
 ('383e47', 53),
 ('decoration', 53),
 ('background', 53),
 ('ffc423', 53),
 ('padding', 53),
 ('10px', 58),
 ('outline', 53),
 ('cursor', 53),
 ('pointer', 53),
 ('2em', 53),
 ('навʼязливої', 53),
 ('жовто', 21),
 ('києво', 31),
 ('клікайте', 49),
 ('title', 22),
 ('аннабелла', 21),
 ('вынтэнэ', 46),
 ('уладзімер', 32),
 ('костанчук', 26),
 ('сингапуре', 24),
 ('арина', 27),
 ('оборота', 23),
 ('нерозважальна', 64),
 ('eugenelakinsky', 80),
 ('пейпел', 85),
 ('емейлу', 79),
 ('u336801545841', 26),
 ('вебмані', 26),
 ('u0456', 53),
 ('u043b', 33),
 ('u043a', 30),
 ('u0441', 31),
 ('u0442', 39),
 ('u0437', 21),
 ('u0430', 54),
 ('u0440', 37),
 ('u0435', 25),
 ('u043e', 63),
 ('u0432', 40),
 (

# Words to indexes, indexes to words dicts

In [20]:
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 158085
Number of words we will use: 116515
Percent of words we will use: 73.7%


# Create word embedding matrix

In [21]:
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

116515


# Convert sentences to sequence of words indexes

In [0]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [23]:
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 1156199
Total number of UNKs in headlines: 52944
Percent of words that are UNK: 4.58%


# Get the length of each sequence

In [0]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [26]:
create_lengths(int_summaries[:3])

,counts
0,13
1,13
2,11


In [27]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
            counts
count  1783.000000
mean     11.479529
std       3.173526
min       2.000000
25%       9.000000
50%      11.000000
75%      13.000000
max      25.000000

Texts:
            counts
count  1783.000000
mean    637.977566
std     546.913226
min       1.000000
25%     203.000000
50%     536.000000
75%     917.500000
max    4219.000000


In [28]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 89.5))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

1310.3400000000006
1638.8999999999999
2431.400000000005


In [29]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

16.0
17.0
19.0


# Count the number of time UNKNOWN appears in a sentence

In [0]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [31]:
max_text_length = 2000 # This will cover up to 89.5% lengthes
max_summary_length = 20 # This will cover up to 99% lengthes
min_length = 2
unk_text_limit = 20 # text can contain up to 1 UNK word
unk_summary_limit = 5 # Summary should not contain any UNK word

def filter_condition(item):
    int_summary = item[0]
    int_text = item[1]
    if(len(int_summary) >= min_length and 
       len(int_summary) <= max_summary_length and 
       len(int_text) >= min_length and 
       len(int_text) <= max_text_length and 
       unk_counter(int_summary) <= unk_summary_limit and 
       unk_counter(int_text) <= unk_text_limit):
        return True
    else:
        return False

int_text_summaries = list(zip(int_summaries , int_texts))
int_text_summaries_filtered = list(filter(filter_condition, int_text_summaries))
sorted_int_text_summaries = sorted(int_text_summaries_filtered, key=lambda item: len(item[1]))
sorted_int_text_summaries = list(zip(*sorted_int_text_summaries))
sorted_summaries = list(sorted_int_text_summaries[0])
sorted_texts = list(sorted_int_text_summaries[1])
# Delete those temporary varaibles
del int_text_summaries, sorted_int_text_summaries, int_text_summaries_filtered
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

1027
1027


# Building the Model

In [0]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [0]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [0]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [0]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [0]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [0]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
    #                                                                _zero_state_tensors(rnn_size, 
    #                                                                                    batch_size, 
    #                                                                                    tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [0]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [0]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [0]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [0]:
epochs = 50
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

# Build graph

In [42]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Graph is built.


# Training the Model

In [43]:
#start = 200000
#end = start + 300000
#sorted_summaries_short = sorted_summaries[start:end]
#sorted_texts_short = sorted_texts[start:end]
sorted_summaries_short = sorted_summaries
sorted_texts_short = sorted_texts
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 2
The longest text length: 1737


In [44]:
per_epoch = 6
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1
update_check

1

In [45]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 6 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Average loss for this update: 21.827
New Record!
Average loss for this update: 8.968
New Record!
Average loss for this update: 7.351
New Record!
Average loss for this update: 7.227
New Record!
Average loss for this update: 7.317
No Improvement.
Average loss for this update: 8.0
No Improvement.
Average loss for this update: 8.003
No Improvement.
Stopping Training.


In [46]:
checkpoint = "best_model.ckpt" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in loaded_graph.as_graph_def().node]
names

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from best_model.ckpt


['input',
 'targets',
 'learning_rate',
 'keep_prob',
 'summary_length',
 'Const',
 'max_dec_len',
 'text_length',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'encoder_0/DropoutWrapperInit/Const',
 'encoder_0/DropoutWrapperInit/Const_1',
 'encoder_0/DropoutWrapperInit_1/Const',
 'encoder_0/DropoutWrapperInit_1/Const_1',
 'encoder_0/bidirectional_rnn/fw/fw/Rank',
 'encoder_0/bidirectional_rnn/fw/fw/range/start',
 'encoder_0/bidirectional_rnn/fw/fw/range/delta',
 'encoder_0/bidirectional_rnn/fw/fw/range',
 'encoder_0/bidirectional_rnn/fw/fw/concat/values_0',
 'encoder_0/bidirectional_rnn/fw/fw/concat/axis',
 'encoder_0/bidirectional_rnn/fw/fw/concat',
 'encoder_0/bidirectional_rnn/fw/fw/transpose',
 'encoder_0/bidirectional_rnn/fw/fw/sequence_length',
 'encoder_0/bidirectional_rnn/fw/fw/Shape',
 'encoder_0/bidirectional_rnn/fw/fw/strided_slice/stack',
 'encoder_0/bidirectional_rnn/fw/fw/strid

# Generating Summaries

In [0]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [48]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random])

checkpoint = "best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', articles.article[random])
print('Original summary:', articles.title[random])#clean_summaries[random]

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from best_model.ckpt
Original Text: Так, міська влада вже не виключає того, що в самому центрі міста можуть з’явитися щонайменше два підземні паркінги — під Михайлівською та Європейською площами. Про це «Українській правді. Київ» недавно розповів радник мера Києва Павло Рябікін. Очевидно, хочуть «прощупати», як на цю божевільну ідею відреагує громадськість. Рябікін бачить лише два шляхи вирішення проблем із паркінгами — впорядкувати наявний паркувальний простір і створити додаткові паркувальні потужності, у тому числі у центрі міста. Про інші способи зменшити кількість автівок радник Кличка чомусь не згадує. Насправді таких альтернатив уже набереться з добрий десяток: це і платний в’їзд у центр, і обмеження за парними/непарними номерними знаками, ускладнення в’їзду великих авто (наприклад, джипів)? пільги для суперміні- та електромобілів тощо. У різному вигляді такі практики чудово зарекомендували себе в багатьох європейських містах — Лондоні, Париж